# [Day 17](https://adventofcode.com/2024/day/17)

In [77]:
from typing import List, Dict, Tuple
import re

def load_program(file_name: str) -> Tuple[Dict[str, int], List[int]]:
    with open(file_name, "r") as f:
        data = f.read().strip()
        
    register = {}
    
    reg_pattern = r"(A|B|C):\s*(\d+)"  # register pattern
    for match in re.finditer(reg_pattern, data):
        reg, num = match.groups()
        register[reg] = int(num)

    prog_pattern = r"Program:\s*([\d,]+)" # program pattern
    numbers = re.findall(prog_pattern, data)

    # Split the numbers by comma and convert to a list of integers
    if numbers:
        program = list(map(int, numbers[0].split(",")))
    return register, program

In [103]:
from typing import Dict, List
from copy import deepcopy


class Computer:

    def __init__(self, register: Dict[str, int]) -> None:
        self.register = deepcopy(register)
        self.ip = None  # instruction pointer
        self.output = ""  # output string

        self.instructions = {
            0: self.adv,
            1: self.bxl,
            2: self.bst,
            3: self.jzn,
            4: self.bxc,
            5: self.out,
            6: self.bdv,
            7: self.cdv,
        }

    def __repr__(self) -> str:
        """
        Return a formatted string representation of all register values.
        """
        output = []
        for key, value in self.register.items():
            output.append(f"Register {key}: {value}")
        return "\n".join(output)

    def run_program(self, program: List[int]):
        # Initialize instruction pointer to the beginning of the program
        self.ip = 0
        # Initialize output string to empty string
        self.output = ""

        # while the instruction pointer is less than the program length
        while self.ip < len(program):
            current_ip = self.ip
            opcode = program[current_ip]  # Read the opcode
            operand = program[current_ip + 1]  # Read the operand

            # Get instruction based on opcode
            instruction = self.instructions[opcode]

            # Perform the instruction
            instruction(operand)

            # Check if self.ip has updated from the current ip.
            if current_ip == self.ip:
                self.ip += 2  # take two steps

        return self.output

    def get_combo_operand(self, operand: int) -> int:
        """
        This helper function ensures that combo operands are dynamically fetched
        from the current register values.
        """
        combo_operands = {
            0: 0,
            1: 1,
            2: 2,
            3: 3,
            4: self.register["A"],
            5: self.register["B"],
            6: self.register["C"],
        }
        return combo_operands[operand]

    def adv(self, operand: int):
        """
        Performs division. The numerator is the value in the A register.
        The denominator is found by raising 2 to the power of the instruction's combo operand.
        The result of the division operation is truncated to an integer and then written to the A register.
        """
        combo_operand = self.get_combo_operand(operand)

        numerator = self.register["A"]
        denominator = 2**combo_operand
        division = numerator // denominator  # Truncate to integer

        # Write value to register A
        self.register["A"] = division

    def bxl(self, operand: int):
        """
        Calculates the bitwise XOR between literal operand and register B and then writes the value to the B register.
        """
        literal_operand = operand
        self.register["B"] = literal_operand ^ self.register["B"]

    def bst(self, operand: int):
        """
        Calculates the value of its combo operand modulo 8 (thereby keeping only its lowest 3 bits), then writes that value to the B register.
        """
        combo_operand = self.get_combo_operand(operand)
        self.register["B"] = combo_operand % 8

    def jzn(self, operand: int):
        """
        Does nothing if the A register is 0. However, if the A register is not zero, it jumps by setting the instruction pointer to the value of its literal operand;
        if this instruction jumps, the instruction pointer is not increased by 2 after this instruction.
        """
        literal_operand = operand
        if self.register["A"]:
            pass  # Do nothing
        else:
            self.ip = literal_operand

    def bxc(self, _):
        """
        Calculates the bitwise XOR of register B and register C, then stores the result in register B.
        """
        self.register["B"] = self.register["B"] ^ self.register["C"]

    def out(self, operand: int):
        """
        Calculates the value of its combo operand modulo 8, then outputs that value.
        """
        combo_operand = self.get_combo_operand(operand)
        value = combo_operand % 8
        if self.output == "":
            self.output = str(value)
        else:
            self.output = self.output + "," + str(value)

    def bdv(self, operand: int):
        """
        Performs division. The numerator is the value in the A register.
        The denominator is found by raising 2 to the power of the instruction's combo operand.
        The result of the division operation is truncated to an integer and then written to the B register.
        """
        combo_operand = self.get_combo_operand(operand)

        numerator = self.register["A"]
        denominator = 2**combo_operand
        division = numerator // denominator  # Truncate to integer

        # Write value to register B
        self.register["B"] = division

    def cdv(self, operand: int):
        """
        Performs division. The numerator is the value in the A register.
        The denominator is found by raising 2 to the power of the instruction's combo operand.
        The result of the division operation is truncated to an integer and then written to the C register.
        """
        combo_operand = self.get_combo_operand(operand)

        numerator = self.register["A"]
        denominator = 2**combo_operand
        division = numerator // denominator  # Truncate to integer

        # Write value to register C
        self.register["C"] = division

## Part 1

In [113]:
register, program = load_program("data.txt")

computer = Computer(register=register)
output = computer.run_program(program=program)
print("Final state of registers:\n" + str(computer))
print("Output from program:", output)

Final state of registers:
Register A: 0
Register B: 0
Register C: 0
Output from program: 0,3,5,4,3,0


## Part 2